In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv
import re
import urllib
from datetime import datetime
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 
r = requests.get(url)
NIT = BeautifulSoup(r.content,"lxml")
type(NIT)

bs4.BeautifulSoup

In [3]:
htmlpage = urllib.request.urlopen(url)
lst = []
for line in htmlpage:
    line = line.strip()
    if re.search('table class', line.decode('utf-8')) :
        lst.append(line)
print(lst)

[b'<table class="wikitable sortable">', b'<table class="multicol" role="presentation" style="border-collapse: collapse; padding: 0; border: 0; background:transparent; width:100%;">', b'<table class="navbox">']


In [5]:
table=NIT.find('table', {'class', 'wikitable sortable'})
headers= [header.text for header in table.find_all('th')]
headers = list(map(lambda x: x.strip(), headers))

rows = []
for row in table.find_all('tr'):
    rows.append([val.text for val in row.find_all('td')])

df1 = pd.DataFrame(rows, columns=headers)
df1 = df1.drop(df1.index[0])
df1=df1[df1.Borough!='Not assigned'].reset_index(drop=True)
df1['Neighbourhood'] = df1['Neighbourhood'].str.replace('\n', '')
foo = lambda a: ",".join(a) 
df=df1.groupby(['Postcode','Borough']).agg({'Neighbourhood': foo}).reset_index()
df.loc[df.Neighbourhood == "Not assigned", 'Neighbourhood'] = df.Borough # 关键句
df.head(10)



,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [6]:
!wget -q -O 'Geospatial_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [7]:
geo_data = pd.read_csv('Geospatial_data.csv')

In [8]:
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
geo_data=geo_data.set_index('Postal Code')
geo_data.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [ ]:
data = df.join(geo_data, on='Postcode')
data.